In [1]:
from pathlib import Path

import pandas as pd
import numpy as np

In [2]:
project_path = Path.cwd()
data_path_unprocessed = project_path / 'data' / '01_reduced'
data_path_preprocessed = project_path / 'data' / '02_preprocessed'
data_path_cleaned = project_path / 'data' / '03_cleaned'
data_path_transformed = project_path / 'data'/ '04_transformed'

In [3]:
df_in = pd.read_parquet(data_path_unprocessed / 'inputs.pqt')
df_out = pd.read_parquet(data_path_unprocessed / 'targets.pqt')

### Cleaning Inputs 

In [4]:
df_in.head()

,id,lat,lon,etopo2,oisst,es411,es443,es489,es510,es555,es670,lw411,lw443,lw489,lw510,lw555,lw670
0,1565,38.4279,-76.61,0.0,3.70,107.9060,121.1870,137.2660,134.0370,130.9470,112.9250,0.129962,0.204331,0.452029,0.541019,0.979378,0.391235
1,1566,38.3680,-76.50,0.0,3.70,113.8600,128.0530,145.1480,140.9650,138.3500,118.5550,0.120886,0.177228,0.315395,0.352306,0.574440,0.200936
2,1567,38.3074,-76.44,1.0,3.70,114.3500,128.0550,146.0600,142.7250,140.1980,119.9780,0.111049,0.151807,0.269218,0.326515,0.595226,0.193438
3,1568,38.6367,-76.32,3.0,3.70,35.0441,38.1511,42.1373,40.5485,39.2907,31.7094,0.051578,0.066430,0.121242,0.148580,0.274316,0.102543
4,1559,38.3047,-76.44,1.0,22.03,61.8732,67.0335,72.2731,69.6545,77.1321,50.0766,0.056001,0.068530,0.108838,0.132581,0.216023,0.089663


In [5]:
λ = [411, 443, 489, 510, 555, 670]
for λi in λ:
    df_in[f'Rrs{λi}'] = df_in[f'lw{λi}'] / df_in[f'es{λi}']

In [6]:
df_in.head()

,id,lat,lon,etopo2,oisst,es411,es443,es489,es510,es555,...,lw489,lw510,lw555,lw670,Rrs411,Rrs443,Rrs489,Rrs510,Rrs555,Rrs670
0,1565,38.4279,-76.61,0.0,3.70,107.9060,121.1870,137.2660,134.0370,130.9470,...,0.452029,0.541019,0.979378,0.391235,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465
1,1566,38.3680,-76.50,0.0,3.70,113.8600,128.0530,145.1480,140.9650,138.3500,...,0.315395,0.352306,0.574440,0.200936,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695
2,1567,38.3074,-76.44,1.0,3.70,114.3500,128.0550,146.0600,142.7250,140.1980,...,0.269218,0.326515,0.595226,0.193438,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612
3,1568,38.6367,-76.32,3.0,3.70,35.0441,38.1511,42.1373,40.5485,39.2907,...,0.121242,0.148580,0.274316,0.102543,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234
4,1559,38.3047,-76.44,1.0,22.03,61.8732,67.0335,72.2731,69.6545,77.1321,...,0.108838,0.132581,0.216023,0.089663,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791


In [7]:
df_in_nonan = df_in.dropna()[['id', 'lat', 'lon', 'etopo2', 'oisst'] + [f'Rrs{λi}' for λi in λ]]

In [8]:
df_in_nonan.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1137 entries, 0 to 4458
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      1137 non-null   int64  
 1   lat     1137 non-null   float64
 2   lon     1137 non-null   float64
 3   etopo2  1137 non-null   float64
 4   oisst   1137 non-null   float64
 5   Rrs411  1137 non-null   float64
 6   Rrs443  1137 non-null   float64
 7   Rrs489  1137 non-null   float64
 8   Rrs510  1137 non-null   float64
 9   Rrs555  1137 non-null   float64
 10  Rrs670  1137 non-null   float64
dtypes: float64(10), int64(1)
memory usage: 106.6 KB


In [9]:
df_in.filter(regex='(id)|(Rrs[0-9]+)', axis=1).to_parquet(data_path_cleaned/'df_in.pqt')

### Cleaning Output

In [10]:
df_out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      4459 non-null   int64  
 1   chl     3392 non-null   float64
 2   chl_a   1381 non-null   float64
 3   ad411   1231 non-null   float64
 4   ad443   1238 non-null   float64
 5   ad489   1237 non-null   float64
 6   ad510   1237 non-null   float64
 7   ap411   1272 non-null   float64
 8   ap443   1279 non-null   float64
 9   ap489   1278 non-null   float64
 10  ap510   1278 non-null   float64
 11  ap555   1262 non-null   float64
 12  ap670   1270 non-null   float64
dtypes: float64(12), int64(1)
memory usage: 453.0 KB


In [11]:
df_out[['chl', 'chl_a']]

,chl,chl_a
0,38.19000,NaN
1,35.01000,NaN
2,26.91000,NaN
3,47.96000,NaN
4,23.55000,NaN
...,...,...
4454,4.62784,NaN
4455,2.95997,NaN
4456,0.95114,NaN
4457,4.54228,NaN


In [12]:
df_chl = df_out[['id', 'chl', 'chl_a']].copy()

In [13]:
def create_chl_and_flag(df):
    """
    Creates 'chl' and 'hplc_flag' columns based on the rules provided.

    Args:
        df (pd.DataFrame): Input DataFrame with 'chl' and 'chl_a' columns.

    Returns:
        pd.DataFrame: DataFrame with new 'chl' and 'hplc_flag' columns.
    """
    new_chl = np.select(
        [
            df['chl_a'].notna(),
            df['chl'].notna() & df['chl_a'].isna(),
            df['chl'].isna() & df['chl_a'].isna()
        ],
        [
            df['chl_a'],
            df['chl'],
            np.nan
        ],
        default=np.nan  # Should not be reached based on the conditions
    )

    hplc_flag = np.select(
        [
            df['chl_a'].notna(),
            df['chl'].notna() & df['chl_a'].isna(),
            df['chl'].isna() & df['chl_a'].isna()
        ],
        ['1', '0', '-1'],
        default='-1' # Should not be reached based on the conditions
    )

    new_df = df.copy()
    new_df['chl'] = new_chl
    new_df['hplc_flag'] = hplc_flag
    return new_df[['id', 'chl', 'hplc_flag']]

In [14]:
df_chl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   id      4459 non-null   int64  
 1   chl     3392 non-null   float64
 2   chl_a   1381 non-null   float64
dtypes: float64(2), int64(1)
memory usage: 104.6 KB


In [15]:
new_df_chl = create_chl_and_flag(df_chl)

In [16]:
new_df_chl.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4459 entries, 0 to 4458
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         4459 non-null   int64  
 1   chl        4127 non-null   float64
 2   hplc_flag  4459 non-null   object 
dtypes: float64(1), int64(1), object(1)
memory usage: 104.6+ KB


In [17]:
new_df_chl.hplc_flag.value_counts()

hplc_flag
0     2746
1     1381
-1     332
Name: count, dtype: int64

In [18]:
df_all = pd.merge(df_in_nonan, new_df_chl, how='inner', left_on='id', right_on='id')

In [19]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1213 non-null   int64  
 1   lat        1213 non-null   float64
 2   lon        1213 non-null   float64
 3   etopo2     1213 non-null   float64
 4   oisst      1213 non-null   float64
 5   Rrs411     1213 non-null   float64
 6   Rrs443     1213 non-null   float64
 7   Rrs489     1213 non-null   float64
 8   Rrs510     1213 non-null   float64
 9   Rrs555     1213 non-null   float64
 10  Rrs670     1213 non-null   float64
 11  chl        1118 non-null   float64
 12  hplc_flag  1213 non-null   object 
dtypes: float64(11), int64(1), object(1)
memory usage: 123.3+ KB


In [20]:
df_all.to_parquet(data_path_cleaned / 'df_all.pqt')

### Transforming model form MBR modeling

In [21]:
df_t = df_all[
    ['id'] 
    + [f'Rrs{i}' for i in λ] 
    + ['chl', 'hplc_flag']].copy()


In [22]:
df_t.loc[:, 'MBR'] = df_t.loc[
    :, [f'Rrs{i}' for i in λ[:-2]]].max(axis=1).divide(
        df_t.loc[:, ['Rrs555', 'Rrs670']].sum(axis=1)
    )
df_t.loc[:, 'MBR_flag'] = df_t.loc[:, [f'Rrs{i}' for i in λ[:-2]]].idxmax(axis=1)

In [23]:
df_t.head()

,id,Rrs411,Rrs443,Rrs489,Rrs510,Rrs555,Rrs670,chl,hplc_flag,MBR,MBR_flag
0,1565,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,38.19,0,0.368826,Rrs510
1,1566,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,35.01,0,0.427444,Rrs510
2,1567,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,26.91,0,0.390537,Rrs510
3,1568,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,47.96,0,0.358694,Rrs510
4,1559,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,23.55,0,0.414576,Rrs510


In [24]:
df_t = df_t[
    [
        'id', 'Rrs411',	'Rrs443', 'Rrs489',	'Rrs510', 'Rrs555',	'Rrs670', 
        'MBR', 'MBR_flag', 'chl', 'hplc_flag'
    ]
]


In [25]:
df_t.head()

,id,Rrs411,Rrs443,Rrs489,Rrs510,Rrs555,Rrs670,MBR,MBR_flag,chl,hplc_flag
0,1565,0.001204,0.001686,0.003293,0.004036,0.007479,0.003465,0.368826,Rrs510,38.19,0
1,1566,0.001062,0.001384,0.002173,0.002499,0.004152,0.001695,0.427444,Rrs510,35.01,0
2,1567,0.000971,0.001185,0.001843,0.002288,0.004246,0.001612,0.390537,Rrs510,26.91,0
3,1568,0.001472,0.001741,0.002877,0.003664,0.006982,0.003234,0.358694,Rrs510,47.96,0
4,1559,0.000905,0.001022,0.001506,0.001903,0.002801,0.001791,0.414576,Rrs510,23.55,0


In [26]:
df_t.hplc_flag.value_counts()

hplc_flag
0     666
1     452
-1     95
Name: count, dtype: int64

In [27]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1213 non-null   int64  
 1   Rrs411     1213 non-null   float64
 2   Rrs443     1213 non-null   float64
 3   Rrs489     1213 non-null   float64
 4   Rrs510     1213 non-null   float64
 5   Rrs555     1213 non-null   float64
 6   Rrs670     1213 non-null   float64
 7   MBR        1213 non-null   float64
 8   MBR_flag   1213 non-null   object 
 9   chl        1118 non-null   float64
 10  hplc_flag  1213 non-null   object 
dtypes: float64(8), int64(1), object(2)
memory usage: 104.4+ KB


In [28]:
df_t.describe().T

,count,mean,std,min,25%,50%,75%,max
id,1213.0,4839.014839,2376.889363,1441.000000,2655.000000,4842.000000,7084.000000,7831.000000
Rrs411,1213.0,0.004553,0.003789,0.000051,0.002132,0.003300,0.006300,0.030600
Rrs443,1213.0,0.004563,0.003410,0.000190,0.002417,0.003516,0.006000,0.027601
Rrs489,1213.0,0.005227,0.003846,0.000367,0.002969,0.004392,0.006000,0.030200
Rrs510,1213.0,0.004988,0.004096,0.000497,0.002854,0.003547,0.005300,0.032900
Rrs555,1213.0,0.005008,0.005033,0.000417,0.001841,0.002901,0.006300,0.046600
Rrs670,1213.0,0.001419,0.002277,0.000000,0.000200,0.000573,0.001700,0.027700
MBR,1213.0,1.703416,1.872232,0.310466,0.663278,0.924495,1.597575,12.445853
chl,1118.0,3.723355,7.444344,0.017000,0.301800,1.048150,3.267068,77.864800


In [29]:
df_t.insert(8, 'log_MBR', np.log10(df_t.MBR))

In [30]:
df_t.insert(11, 'log_chl', np.log10(df_t.chl))

In [31]:
df_t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1213 entries, 0 to 1212
Data columns (total 13 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         1213 non-null   int64  
 1   Rrs411     1213 non-null   float64
 2   Rrs443     1213 non-null   float64
 3   Rrs489     1213 non-null   float64
 4   Rrs510     1213 non-null   float64
 5   Rrs555     1213 non-null   float64
 6   Rrs670     1213 non-null   float64
 7   MBR        1213 non-null   float64
 8   log_MBR    1213 non-null   float64
 9   MBR_flag   1213 non-null   object 
 10  chl        1118 non-null   float64
 11  log_chl    1118 non-null   float64
 12  hplc_flag  1213 non-null   object 
dtypes: float64(10), int64(1), object(2)
memory usage: 123.3+ KB


In [34]:
df_t.to_parquet(data_path_transformed/'df_all.pqt')